# Set up session

In [1]:
spark

In [2]:
spark.sql("SET spark.sql.shuffle.partitions = 1024")
spark.sql("SET hive.exec.dynamic.partition.mode = nonstrict")  # necessary for dynamic wiki_db partition

DataFrame[key: string, value: string]

In [3]:
import re

# Build Outlinks Dataset

In [4]:
snapshot = '2020-05'  # data will be current to this date -- e.g., 2020-05 means data is up to 30 April 2020 (at least)
table_name = 'isaacj.outlinks_allwikis_202005'  # Hive table where data will be inserted

## Create external table (won't be deleted by accident in Hive) for data
You only have to do this once but re-running it won't overwrite data

In [5]:
create_table_query = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {0} (
        qid_from     STRING  COMMENT 'Wikidata item ID of source Wikipedia article (e.g., Q42)',
        pid_from     INT     COMMENT 'Wikipedia page ID of source Wikipedia article (e.g., 8091)',
        qid_to       STRING  COMMENT 'Wikidata item ID of target Wikipedia article (e.g., Q3107329)',
        pid_to       INT     COMMENT 'Wikipedia page ID of target Wikipedia article (e.g., 478921)'
    )
    PARTITIONED BY (
        wiki_db      STRING  COMMENT 'Wikipedia languauge edition (e.g., enwiki)'
    )
    STORED AS PARQUET
    LOCATION 'hdfs://analytics-hadoop/user/isaacj/lang-agnostic'
    """.format(table_name)

spark.sql(create_table_query)

DataFrame[]

## Compile data and insert it
* Processing:
 * Only Wikipedia wikis (via `hostname like '%wikipedia%'` in `wikipedia_projects` and subsequent inner joins)
 * Resolve redirects -- e.g., Barack Obama -> Chicago, Illinois (redirect) -> Chicago becomes Barack Obama -> Chicago
 * Join in Wikidata items -- this is a left join so pages without Wikidata items are retained

In [6]:
print_for_hive = False
do_execute = False

query = """
WITH wikipedia_projects AS (
        SELECT dbname
          FROM wmf_raw.mediawiki_project_namespace_map
         WHERE snapshot = '{0}'
               AND hostname LIKE '%wikipedia%'
        ),
title_to_id AS (
    SELECT page_id,
           page_title,
           wiki_db
      FROM wmf_raw.mediawiki_page mp
     INNER JOIN wikipedia_projects wp
           ON (mp.wiki_db = wp.dbname)
     WHERE page_namespace = 0
           AND snapshot = '{0}'
),
redirects AS (
    SELECT mr.rd_from AS rd_from,
           tti.page_id AS rd_to,
           mr.wiki_db AS wiki_db
      FROM wmf_raw.mediawiki_redirect mr
     INNER JOIN title_to_id tti
           ON (mr.rd_title = tti.page_title
               AND mr.wiki_db = tti.wiki_db)
     WHERE mr.snapshot = '{0}'
           AND mr.rd_namespace = 0
),
pagelinks_reformatted AS (
    SELECT pl.pl_from AS pl_from,
           tti.page_id AS pl_to,
           pl.wiki_db AS wiki_db
      FROM wmf_raw.mediawiki_pagelinks pl
     INNER JOIN title_to_id tti
           ON (pl.pl_title = tti.page_title
               AND pl.wiki_db = tti.wiki_db)
      LEFT ANTI JOIN redirects r
           ON (pl.pl_from = r.rd_from
               AND pl.wiki_db = r.wiki_db)
     WHERE snapshot = '{0}'
           AND pl_from_namespace = 0
           AND pl_namespace = 0
),
pagelinks_redirects_resolved AS (
    SELECT DISTINCT pl.pl_from AS pl_from,
           COALESCE(r.rd_to, pl.pl_to) AS pl_to,
           pl.wiki_db AS wiki_db
      FROM pagelinks_reformatted pl
      LEFT JOIN redirects r
           ON (pl.pl_to = r.rd_from
               AND pl.wiki_db = r.wiki_db)
),
exploded_wikidata_links AS (
    SELECT wiki_db,
           page_id,
           FIRST_VALUE(item_id, true) OVER (PARTITION BY wiki_db, page_id ORDER BY snapshot DESC) as item_id
      FROM wmf.wikidata_item_page_link
     WHERE snapshot >= '2020-05-18'
           AND page_namespace = 0
    ),
wikidata_ids AS (
    SELECT DISTINCT wd.wiki_db AS wiki_db,
           wd.page_id AS page_id,
           wd.item_id AS item_id
      FROM exploded_wikidata_links wd
     INNER JOIN wikipedia_projects p
           ON (wd.wiki_db = p.dbname)
    )
INSERT OVERWRITE TABLE {1}
PARTITION(wiki_db)
SELECT wf.item_id AS qid_from,
       p.pl_from AS pid_from,
       wt.item_id AS qid_to,
       p.pl_to AS pid_to,
       p.wiki_db as wiki_db
  FROM pagelinks_redirects_resolved p
  LEFT JOIN wikidata_ids wf
       ON (p.pl_from = wf.page_id
           AND p.wiki_db = wf.wiki_db)
  LEFT JOIN wikidata_ids wt
       ON (p.pl_to = wt.page_id
           AND p.wiki_db = wt.wiki_db)
""".format(snapshot, table_name)

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    result = spark.sql(query)


WITH wikipedia_projects AS (
        SELECT dbname
          FROM wmf_raw.mediawiki_project_namespace_map
         WHERE snapshot = '2020-05'
               AND hostname LIKE '%wikipedia%'
        ),
title_to_id AS (
    SELECT page_id,
           page_title,
           wiki_db
      FROM wmf_raw.mediawiki_page mp
     INNER JOIN wikipedia_projects wp
           ON (mp.wiki_db = wp.dbname)
     WHERE page_namespace = 0
           AND snapshot = '2020-05'
),
redirects AS (
    SELECT mr.rd_from AS rd_from,
           tti.page_id AS rd_to,
           mr.wiki_db AS wiki_db
      FROM wmf_raw.mediawiki_redirect mr
     INNER JOIN title_to_id tti
           ON (mr.rd_title = tti.page_title
               AND mr.wiki_db = tti.wiki_db)
     WHERE mr.snapshot = '2020-05'
           AND mr.rd_namespace = 0
),
pagelinks_reformatted AS (
    SELECT pl.pl_from AS pl_from,
           tti.page_id AS pl_to,
           pl.wiki_db AS wiki_db
      FROM wmf_raw.mediawiki_pagelinks pl
     INNER JOIN ti

In [8]:
outlink_counts_query = """
SELECT wiki_db,
       COUNT(1) AS num_outlinks,
       COUNT(DISTINCT(pid_from)) AS num_unique_src,
       COUNT(DISTINCT(qid_from)) AS num_src_with_qids,
       COUNT(DISTINCT(pid_to)) AS num_unique_trgt,
       COUNT(DISTINCT(qid_to)) AS num_trgt_with_qids
  FROM {0}
 GROUP BY wiki_db
 ORDER BY wiki_db
 LIMIT 1000
""".format(table_name)
spark.sql(outlink_counts_query).show(n=500)

+----------------+------------+--------------+-----------------+---------------+------------------+
|         wiki_db|num_outlinks|num_unique_src|num_src_with_qids|num_unique_trgt|num_trgt_with_qids|
+----------------+------------+--------------+-----------------+---------------+------------------+
|          abwiki|      212325|          6000|             5902|           5513|              5484|
|         acewiki|       56114|         10265|            10229|           2512|              2488|
|         adywiki|        1812|           390|              390|            365|               365|
|          afwiki|     3068481|         91279|            91228|          62956|             62926|
|          akwiki|        2346|           698|              691|            199|               189|
|         alswiki|      908379|         27144|            27126|          25865|             25845|
|          amwiki|      111655|         14686|            14604|          11420|             11324|
